In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

data = pd.read_csv('datasets/Dataset1/Dataset1.csv')

In [ ]:
# Contoh: label encode kolom churn
le = LabelEncoder()
data['Churn'] = le.fit_transform(data['Churn'])

# Memisah fitur dan target
y = data['Churn']
X = data.drop('Churn', axis=1)

# One-hot encode categoricals
X = pd.get_dummies(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Scaling
tmp = StandardScaler()
X_train = tmp.fit_transform(X_train)
X_test = tmp.transform(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svm = SVC(kernel='rbf', probability=True)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=200, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Ubah X_train/X_test ke bentuk 3D
timesteps = X_train.shape[1]  # jumlah fitur
X_train_cnn = X_train.reshape(-1, timesteps, 1)
X_test_cnn = X_test.reshape(-1, timesteps, 1)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(timesteps, 1)),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_cnn, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)

loss, acc = model.evaluate(X_test_cnn, y_test_cat)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Contoh AUC SVM
y_prob = svm.predict_proba(X_test)[:,1]
auc_svm = roc_auc_score(y_test, y_prob)
print(f"SVM ROC-AUC: {auc_svm:.4f}")